In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv('kpmg_encoded.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20495 entries, 0 to 20494
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   past_3_years_bike_related_purchases  20495 non-null  int64  
 1   owns_car                             20495 non-null  int64  
 2   tenure                               20495 non-null  float64
 3   property_valuation                   20495 non-null  float64
 4   online_order                         20495 non-null  float64
 5   order_status                         20495 non-null  int64  
 6   list_price                           20495 non-null  float64
 7   standard_cost                        20495 non-null  float64
 8   age                                  20495 non-null  float64
 9   F                                    20495 non-null  int64  
 10  M                                    20495 non-null  int64  
 11  Affluent Customer           

In [4]:
#selecting the dependent and independent variables

x = data.loc[:, data.columns != 'order_status']
y = data.iloc[:,5].values